# The second example

from here: https://quantumcomputing.stackexchange.com/questions/15781/hhl-algorithm-how-can-i-get-result-from-register-b-rangle

In [1]:
# https://quantumcomputing.stackexchange.com/questions/24050/how-to-implement-a-exponential-of-a-hamiltonian-but-non-unitary-matrix-in-qisk

# control gate: https://qiskit.org/documentation/stubs/qiskit.circuit.ControlledGate.html

# circuit https://www.nature.com/articles/s41598-022-17660-8
import numpy as np

from qiskit.circuit import ControlledGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer, transpile
from qiskit.quantum_info.operators import Operator
from qiskit.extensions import UnitaryGate
from qiskit.tools.visualization import plot_histogram

from qiskit.synthesis import MatrixExponential
from qiskit.quantum_info import Operator, DensityMatrix, Statevector
from scipy.stats import unitary_group, ortho_group  

In [2]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.opflow.list_ops import SummedOp
from qiskit.circuit import Parameter
from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki, PauliSumOp
A = ortho_group.rvs(2)
b = np.array([0, 1]).T
print(f'the matrix A is {A}')
x = np.linalg.solve(A,b)
print('x-vector', x)
print(np.sum(x**2))

the matrix A is [[-0.94470676  0.32791635]
 [-0.32791635 -0.94470676]]
x-vector [-0.32791635 -0.94470676]
0.9999999999999999


In [3]:
def get_gate(A, n):    
    pauli_op = PauliSumOp(SparsePauliOp.from_operator(A))
    # phi = Parameter('ϕ')
    # evolution_op = (phi * pauli_op).exp_i() # exp(-iϕA)
    # trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(evolution_op).bind_parameters({phi: np.pi/n})
    #----control---------
    gate = evolution_op.to_circuit()
    gate.name = f"A"
    gate.label = f"A"
    gate = gate.to_gate().control()
    #---------------------
    return gate

In [4]:
n_b = 1
n_ = 12
n_ancilla = 1
n_cl = 2
# quantum circuit initialization
qc = QuantumCircuit(n_b + n_ + n_ancilla, 1)
# b-vector state preparation
for i in range(n_b):
    qc.x(i)
for i in range(n_b, n_b+n_):
    qc.h(i)
qc.barrier()
# Matrix exponentiation
for i in range(0, n_):
    gate = get_gate(-A, 2**(i+1))
    qc.append(gate,[i+1, 0])
    qc.crz(-np.pi/float(2**(i+1)), i+1, 0)
qc.barrier()
# # # Phase estimation
for j in range(n_b + n_ - 1, n_b, -1):
    qc.h(j)
    for m in range(j - n_b):
        qc.crz(-np.pi/float(2**(j-m - n_b)), j, m+n_b)
qc.h(j-1)
qc.barrier()
# # As I understood, we wncode ancilla qubit to be sure that result will be correct
for j in range(1, 1+n_):
    qc.cry(np.pi/(2**j), n_b+n_-j, n_b+n_)
qc.barrier()
# # Inverse quantum Fourier transform
for j in range(n_b, n_b + n_):
    for m in range(j - n_b):
        qc.crz(np.pi/float(2**(j-m - n_b)), j, m+n_b)
    qc.h(j)
qc.h(n_b)
qc.barrier()
# # Eigenvalues storing in the vecor b register
for i in range(n_, 0, -1):
    gate = get_gate(A, -(2**(i)))
    qc.append(gate,[i, 0])
    qc.crz(np.pi/float(2**(i)), i, 0)
qc.barrier()
# qubits measurement. I do not measure the ancilla qubit
# print(np.round(Operator.from_circuit(qc), 3))
# np.round(DensityMatrix.from_instruction(qc).data, 2)
vec = Statevector.from_instruction(qc).data
# print(np.round(vec, 2))
vec = np.array([[vec[i]] for i in range(len(vec))])
# print(vec)
rho = np.dot(vec, vec.T)
# print(rho)
# DensityMatrix.from_instruction(qc).data
# print(np.round(rho,2))
# print(np.round(rho.T, 2))
# qc.draw(output='mpl', style={'backgroundcolor': '#EEEEEE'})

ValueError: Operator contains complex coefficients, which are not supported.

In [ ]:
# qc.measure(-1, 1)
qc.measure(0, 0)
# qc.draw(output='mpl', style={'backgroundcolor': '#EEEEEE'})

In [36]:
simulator = Aer.get_backend('qasm_simulator')
circ = transpile(qc, simulator)
shots = 10000
result = execute(qc, backend=simulator, shots=shots).result()

# Calculated result

In [37]:
counts = result.get_counts()
probabilities = counts.copy()
for k, v in probabilities.items():
    probabilities[k] = v/shots
# print(probabilities)
vals = np.array(list(probabilities.values()))
print(vals)

[0.854 0.146]


# Result x-vector from paper

In [38]:
x = np.linalg.solve(A,b)
print(f"real x {x},\n sqrt {np.sqrt(np.abs(x))},\n squared {x**2},\n normed {np.array(x) / np.linalg.norm(x)},\n"
      f"normed squarsed{(np.array(x) / np.linalg.norm(x))**2}")

real x [-0.13761376  0.99048597],
 sqrt [0.37096328 0.99523162],
 squared [0.01893755 0.98106245],
 normed [-0.13761376  0.99048597],
normed squarsed[0.01893755 0.98106245]


In [ ]:
{'0': 0.4469, '1': 0.5531}

In [69]:
np.sum(np.sqrt(x)**2)

(-0.28723930907958567-1.3266442186877667j)

# Normalization of real x-vector result

In [70]:
a = np.linalg.solve(A,b)

from sklearn.preprocessing import normalize
norm1 = np.array(a) / np.linalg.norm(a)
np.sqrt(a)

array([0.4772977 -0.75015003j, 0.57449345-0.53138504j])

In [ ]:
{'0': 0.4469, '1': 0.5531}